In [92]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
from tensorflow.keras.datasets import mnist
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten


In [93]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [94]:
IMG_SIZE = 32

In [95]:
# resize images for vgg19
import cv2

def resize(img_array):
    tmp = np.empty((img_array.shape[0], IMG_SIZE, IMG_SIZE))

    for i in range(len(img_array)):
        img = img_array[i].reshape(28, 28).astype('uint8')
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img.astype('float32')/255
        tmp[i] = img
        
    return tmp

train_x_resize = resize(X_train)
test_x_resize = resize(X_test)

In [96]:
train_x_final = np.stack((train_x_resize,)*3, axis=-1)
test_x_final = np.stack((test_x_resize,)*3, axis=-1)
print(train_x_final.shape)
print(test_x_final.shape)

(60000, 32, 32, 3)
(10000, 32, 32, 3)


In [97]:
train_y_final = to_categorical(y_train, num_classes=10)
print(train_y_final.shape)

(60000, 10)


In [98]:
X_train, X_test, y_train, y_test = train_test_split(train_x_final, train_y_final, test_size=0.2, random_state=2019)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(48000, 32, 32, 3)
(12000, 32, 32, 3)
(48000, 10)
(12000, 10)


In [99]:
vgg19 = VGG19(weights = 'imagenet', 
              include_top = False,
              input_shape=(IMG_SIZE, IMG_SIZE, 3)
              )

model = Sequential()
model.add(vgg19)
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy',
              tf.keras.metrics.Precision(0.5),
              tf.keras.metrics.Recall(0.5),])

model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 1, 1, 512)         20024384  
_________________________________________________________________
flatten_9 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 10)                5130      
Total params: 20,029,514
Trainable params: 20,029,514
Non-trainable params: 0
_________________________________________________________________


In [100]:
earl = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')

In [101]:
history = model.fit(X_train, 
                    y_train, 
                    epochs=5, 
                    batch_size=32,
                    validation_data=(X_test,y_test), 
                    callbacks=[earl])

Epoch 1/5
1500/1500 [==============================] - 59s 38ms/step - loss: 0.6725 - accuracy: 0.7638 - precision_6: 0.9053 - recall_6: 0.6925 - val_loss: 0.1897 - val_accuracy: 0.9564 - val_precision_6: 0.9657 - val_recall_6: 0.9482
Epoch 2/5
1500/1500 [==============================] - 58s 39ms/step - loss: 0.1437 - accuracy: 0.9678 - precision_6: 0.9743 - recall_6: 0.9629 - val_loss: 0.1242 - val_accuracy: 0.9709 - val_precision_6: 0.9800 - val_recall_6: 0.9648
Epoch 3/5
1500/1500 [==============================] - 61s 40ms/step - loss: 0.1561 - accuracy: 0.9636 - precision_6: 0.9726 - recall_6: 0.9569 - val_loss: 0.0903 - val_accuracy: 0.9793 - val_precision_6: 0.9839 - val_recall_6: 0.9771
Epoch 4/5
1500/1500 [==============================] - 61s 40ms/step - loss: 0.0838 - accuracy: 0.9805 - precision_6: 0.9852 - recall_6: 0.9770 - val_loss: 0.0761 - val_accuracy: 0.9824 - val_precision_6: 0.9862 - val_recall_6: 0.9797
Epoch 5/5
1500/1500 [==============================] - 60s 4

In [102]:
score = model.evaluate(X_test, y_test)
score_df = pd.Series(score, index = model.metrics_names)
score_df

375/375 [==============================] - 4s 11ms/step - loss: 0.0727 - accuracy: 0.9839 - precision_6: 0.9893 - recall_6: 0.9802


loss           0.072683
accuracy       0.983917
precision_6    0.989318
recall_6       0.980167
dtype: float64

In [103]:
model.save('model/vgg19_mnist')

INFO:tensorflow:Assets written to: model/vgg19_mnist\assets


In [104]:
model.save_weights('weights/vgg19_weights')